# ECEN 4322-5322 Data and Network Science

## Title- Fare prediction for flights


#### Group Members - Chirag Chandrashekar, Chris Alexander, Viveka Salinamakki

## Introduction 

The dataset chosen for analysis is the itineraries of flights in the USA over **6 months**. For exploratory data analysis, we aim to find the airport with the highest traffic or the best-connected airports, the price of flights during the different hours of the day, the average/minimum travel distance for which people prefer flight, whether an average flight is fully booked/percentage of flights booked, and popular airlines. The end goal here is to predict the fare of a flight. Due to the large number of rows and features, the prediction can provide a good estimation of the fare. Techniques such as data grouping and manipulation, visualization, regular expressions, data modeling, feature engineering, model validation, and prediction will help achieve our goal. 


<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

## Importing the Data

In [ ]:
import numpy as np

import pandas as pd
from pandas.api.types import CategoricalDtype

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

import zipfile
import os

from ecen5322_utils import run_linear_regression_test

# Plot settings
plt.rcParams['figure.figsize'] = (12, 9)
plt.rcParams['font.size'] = 12

### Sampling the Dataset

The origianl data file is sampled due to its size of **30gb** not able to be read on to the ram. Hence, this is solved by reading the file into the chunks of 1million entries and sampling all the chunks to get a final dataset of **500 thousand entries**.

In [ ]:
'''
dfs=[]
with pd.read_csv("car_price_prediction.csv", chunksize=1000000) as reader: #read as chunks due to low system resorces to read 30gb file
    reader
    for chunk in reader:
        #print(type(chunk))
        data_index=chunk.index
        shuffled_indices = np.random.permutation(data_index)  #shuffling and sampling data to reduce the data to 500000 entries
        #print(chunk.loc[shuffled_indices])
        #print(type(chunk))
        chunk,leftover= np.split(chunk.loc[shuffled_indices],[12000]) #selects first N rows from each chunk
        #print(type(chunk))
        #print(chunk)
        #chunks=chunk.to_frame
        #print(chunks)
        dfs.append(chunk) #makes a list of dataframe chunks
        #joined_chunk=pd.concat(chunk)
        #print(joined_chunk)
final_df=pd.concat(dfs) #concats all the chunks in the list of dataframe
print("final dataframe")
final_df
'''

In [ ]:
#Load data to new csv file
#final_df
'''
data_index2=final_df.index
shuffled_indices2 = np.random.permutation(data_index2)
final_df2,leftover2=np.split(final_df.loc[shuffled_indices2],[500000])
final_df2
'''

In [ ]:
#from google.colab import  files
#final_df2.to_csv('sampled_file.csv')
#files.download('sampled_file.csv')

Importing the sampled csv data file

In [ ]:
#data=pd.read_csv('sampled_file.csv')

#New read_csv function to read empty values as -1 and remove unamed column
data=pd.read_csv("sampled_file.csv"
                 #, dtype=str
                 , keep_default_na=True
                 , na_values=-1
                 , na_filter=True).drop(columns=['Unnamed: 0'])
data

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

## Cleaning the dataset

### Drop rows with values 'NaN'

The rows with the values **'NaN'** will be dropped except for the description.

In [ ]:
data = data.dropna(subset=['totalTravelDistance'])
print(data.iloc[:,0:28].isna().sum())

### Function to change True and False to 1 and 0

The columns with values True and False will be changed to 1 and 0 which will be better to train a model.

In [ ]:
def map_true_false(df, columnName):
    """
        Inputs:
            df [Dataframe]: Dataframe on which the operation is performed
            columnName [String]: The column which is being modified
        
        Output:
            New Dataframe with the modified
    """
    df[columnName]=df[columnName].map(dict({True: 1, False: 0}))
    return df

data=map_true_false(data,'isBasicEconomy')
data=map_true_false(data,'isRefundable')
data=map_true_false(data,'isNonStop')
data

### Function to remove outliers

`remove_outliers` function removes the outliers depending on the given condition.

In [ ]:
def remove_outliers(data, variable, lower=-np.inf, upper=np.inf):
  
    """
    Input:
      data (data frame): the table to be filtered
      variable (string): the column with numerical outliers
      lower (numeric): observations with values lower than this will be removed
      upper (numeric): observations with values higher than this will be removed
    
    Output:
      a data frame with outliers removed
      
    Note: This function should not change mutate the contents of data.
    """  
    data_2=data[data[variable]<= upper]
    data_3=data_2[data_2[variable]>= lower]
    return data_3

In [ ]:
data = remove_outliers(data, 'baseFare', lower = 60)
data.loc[:,'baseFare'].sort_values()

### rename columns

In [ ]:
data.columns

### Convert date time column to date time format

The dates are converted to datetime format using `convertToDateTimeFormat` so that the year, month and day will be easily accessible

In [ ]:
def convertToDateTimeFormat(df, columnName):
    """
    Inputs:
        df [Dataframe]: Dataframe on which the operation is performed
        columnName [String]: The column which has dates to be converted into datetime format
                        Acceptable names are 'searchDate' and 'flightDate'
        
    Output:
        New Dataframe with dates converted into datetime format
    """
    df[columnName]=pd.to_datetime(df[columnName], format="%Y-%m-%d")
    return df

In [ ]:
data=convertToDateTimeFormat(data, 'searchDate')
data=convertToDateTimeFormat(data, 'flightDate')

In [ ]:
data['searchDate']

### Create columns for month and day for the date columns

The month and year are extracted from the date columns using `createMonthDay` so that they more usable

In [ ]:
def createMonthDay(df, columnName):
    """
    Inputs:
        df [Dataframe]: Dataframe on which the operation is performed
        columnName [String]: The column which has dates from which month and year are extracted
                        Acceptable names are 'searchDate' and 'flightDate'
        
    Output:
        New Dataframe with with the columns for month and year added from the the column columnName
    """
    df[columnName+'_month']=df[columnName].dt.month
    df[columnName+'_day']=df[columnName].dt.day
    return df

In [ ]:
data=createMonthDay(data, 'searchDate')
data=createMonthDay(data, 'flightDate')

In [ ]:
data

### Extracting duration of the flight in hours

Current format of the time is in as string form with other character, Eg: `PT5H17M` which is 5 hours and 17 mins. the time is extracted using regex and saved into a new `Flight_duration` column in hours.

In [ ]:
def extract_travel_duration(dataFrame, columnName):
    """
    Inputs:
        df [Dataframe]: Dataframe on which the operation is performed
        columnName [String]: The column which is being modified
        
    Output:
        New Dataframe with the modified
    """
    time_columns = pd.DataFrame()
    hour_segment=r"(\d+)H"
    min_segment=r"(\d+)M"
    time_columns["Hour"]=dataFrame[columnName].str.extract(hour_segment).fillna(0).astype(int)
    time_columns["Min"]=dataFrame[columnName].str.extract(min_segment).fillna(0).astype(int)
    dataFrame["Flight_duration"] = time_columns["Hour"]+(time_columns["Min"]/60)
    return dataFrame
    
data=extract_travel_duration(data, "travelDuration")
data["Flight_duration"]

### Extract segments using regex 

The data has many columns with data about different airports, airlines, etc., and the flights with layovers have all these data in the same column. We are using regular expressions to extract the different segments and separate them into different columns.

For example: The airport codes `ATL||JFK` will be split into `ATL` and `JFK` and added under different columns.

In [ ]:
def extractSegments(dataFrame, columnName):
    """
    Inputs:
        df [Dataframe]: Dataframe on which the operation is performed
        columnName [String]: The column which is being modified
        
    Output:
        New Dataframe with the modified
    """
    #Pattern for seperating the segments
    segmentPattern=r"([^||]+)"
   
    #Create new columns for the extracted segments
    dataFrame[columnName+"1"]=dataFrame[columnName].str.findall(segmentPattern).str[0]
    dataFrame[columnName+"2"]=dataFrame[columnName].str.findall(segmentPattern).str[1]
    dataFrame[columnName+"3"]=dataFrame[columnName].str.findall(segmentPattern).str[2]
    return dataFrame

columnsWithSegments=[iterator for iterator in data if iterator.startswith('segments')]

for i in columnsWithSegments:
    data=extractSegments(data, i)

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

## Exploratory Data Analysis

### Distribution of Base Fare

We plot a histogram for base fare to see its distribution. This distribution helps us understand how the values are spread apart and helps find outliers present in the data. 

In [ ]:
sns.histplot(data['baseFare'],color="blue",kde=True,label='baseFare')
plt.title("Histogram of BasePrice column")

From the plot it seems like most of the entries base price are in the range of 0-1500. values above 1500 are not outliers but  

### Best Connected Airport (Airport with the highest traffic)

The best connected data will be the one which has the highest amount of traffic passing through it. The best connected airport will be found out using the arrival and departure data given in the dataframe.

The layovers will have redundant data i.e., the airport will be repeated in the arrival and departure columns which will be taken into account.

In [ ]:
airportCodesSeries=data['segmentsArrivalAirportCode1'].append(data['segmentsArrivalAirportCode2']).append(data['segmentsArrivalAirportCode3']).append(data['segmentsDepartureAirportCode1'])
airportCodesSeries.value_counts().head(5)

In [ ]:
Height_plot=np.array(airportCodesSeries.value_counts().head(5))
plt.bar(x=airportCodesSeries.value_counts().head(5).index,height=Height_plot,color=['orange', 'red', 'green', 'purple', 'pink'])
plt.title("Top 5 best connected airports")
plt.xlabel("Airports")
plt.ylabel("Count")

The **Chicago, IL O’Hare (ORD)** airport is the best connected airport meaning that the highest amount of domestic air traffic goes through this airport compared to other airports in the United States.

### Fare price for different flight duration

Plotting a graph of Fare price vs flight duration will give an understanding of its trend and any correlation which might be present between the features.

In [ ]:
sns.lineplot(x=data["Flight_duration"], y=data["baseFare"])
plt.title("Basefare vs Flight duration")

In [ ]:
#condition=data[data["Flight_duration"]>=2.5 and data["Flight_duration"]<=12.5]
#sns.lineplot(x=condition["Flight_duration"], y=data["baseFare"])

From the above plot there seems to be a linear relationship between flight duration and baseFare in the range 2 to 15 hour flights

### The minimum and average distance for which people prefer flights

The distance travelled in each travel is calculated using segmentDistance columns and so the distance for which people prefer flights can be analyzed.

In [ ]:
data.loc[:,['segmentsDistance','segmentsDistance1', 'segmentsDistance2', 'segmentsDistance3']]

In [ ]:
data['segmentsDistance1']=data['segmentsDistance1'].fillna(0).astype(int)
data['segmentsDistance2']=data['segmentsDistance2'].fillna(0).astype(int)
data['segmentsDistance3']=data['segmentsDistance3'].fillna(0).astype(int)
data['totalDistance']=data['segmentsDistance1']+data['segmentsDistance2']+data['segmentsDistance3']

In [ ]:
data['totalDistance'].describe(), data['totalDistance'].median()

The minimum distance travelled using domestic flights is **89 miles** and a median distance covered by a domestic flight is **1482 miles**.

### whether the average flight is fully booked /average% of seat booked- Chirag

In [ ]:
dataSegment1 = data.groupby('segmentsAirlineName1', as_index=False)['seatsRemaining'].sum()
dataSegment2 = data.groupby('segmentsAirlineName2', as_index=False)['seatsRemaining'].sum()
dataSegment3 = data.groupby('segmentsAirlineName3', as_index=False)['seatsRemaining'].sum()
dataSegment1['averageSeatsRemaining'] = (dataSegment1['seatsRemaining']/len(data.index)).round(0)
print(dataSegment1.sort_values(by = 'seatsRemaining', ascending = False))


### top 5 most popular airlines- Chirag

In [ ]:
airlineName=data['segmentsAirlineName1'].append(data['segmentsAirlineName2']).append(data['segmentsAirlineName3']).append(data['segmentsAirlineName1'])
airlineName.value_counts().head(5)

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

## Prediction of fare

### Feature engineering

### One hot encoding for categorical data

The categorical data present cannot be used directly in the prediction as they do not contain measurable data. The data is hence converted into features using `OneHotEncoder` from `sklearn`.

The columns identified to be OneHotEncoded are:
`startingAirport`, `destinationAirport`, `segmentsArrivalAirportCode`, `segmentsDepartureAirportCode`, `segmentsAirlineName`, `segmentsAirlineCode`, `segmentsCabinCode`.

In these columns, `segmentsAirlineName` and `segmentsAirlineCode` convey the same information and therefore redundant. One of the 2 columns is suffiecient and we have chosen `segmentsAirlineCode`.

The data of `startingAirport` and `destinationAirport` are already included in `segmentsArrivalAirportCode` and `segmentsDepartureAirportCode` and so are redundant information.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
def oheCabinCodeColumns(oheData):
    """
        Inputs:
            oheData [Dataframe]: Dataframe on which the operation is performed
        
        Output:
            Dataframe with the OneHotEncoded features from the column 'segmentsCabinCode'
    """
    
    #columnName is the column to be OneHotEncoded
    columnName='segmentsCabinCode'
    
    #The column 'segmentsCabinCode' has been split into 3 different columns (the layover data) and so all the 3 columns have to be OneHotEncoded
    oneHotEnc = OneHotEncoder(dtype=int, handle_unknown='ignore')
    oheDataColumn3 = oneHotEnc.fit_transform(oheData[[columnName+'3']]).toarray()
    numberOfColumns3=np.shape(oheDataColumn3)[1]
    oheDataColumn2 = oneHotEnc.fit_transform(oheData[[columnName+'2']]).toarray()
    numberOfColumns2=np.shape(oheDataColumn2)[1]
    oheDataColumn1 = oneHotEnc.fit_transform(oheData[[columnName+'1']]).toarray()
    numberOfColumns1=np.shape(oheDataColumn1)[1]
    
    #The minimum number of columns are selected because the segment2 and segment3 have NaNs which are absent in segment1 and also unnecessary to the prediction
    numberOfColumns=min(numberOfColumns3, numberOfColumns2, numberOfColumns1)
    
    #The OR operation is done as the features from the 3 columns are the data of the same categories and so a common set of columns for all the 3 segments will suffice
    oheData[oneHotEnc.categories_[0]] = oheDataColumn1[:,:numberOfColumns] | oheDataColumn2[:,:numberOfColumns] | oheDataColumn3[:,:numberOfColumns]
    return oheData

In [ ]:
#data=oheCabinCodeColumns(data)

In [ ]:
def oheSegmentsAirportCode_AirlineName(oheData, columnName):
    """
        Inputs:
            oheData [Dataframe]: Dataframe on which the operation is performed
            columnName [String]: The column name (only the ones mentioned below) of the column to be OneHotEncoded
                                 Acceptable columnNames are 'segmentsArrivalAirportCode', 'segmentsDepartureAirportCode', or 'segmentsAirlineCode'
        
        Output:
            Dataframe with the OneHotEncoded features from the column columnName
    """

    oneHotEnc = OneHotEncoder(dtype=int, handle_unknown='ignore')
    
    #The acceptable columnName columns have been split into 3 different columns (the layover data) and so all the 3 columns have to be OneHotEncoded
    #dfIntermediate dataframes are to get the features of each of the segments individually
    dfIntermediate3=pd.DataFrame()
    dfIntermediate2=pd.DataFrame()
    dfIntermediate1=pd.DataFrame()
    oheDataColumn3 = oneHotEnc.fit_transform(oheData[[columnName+'3']]).toarray()
    dfIntermediate3[oneHotEnc.categories_[0]]=oheDataColumn3
    oheDataColumn2 = oneHotEnc.fit_transform(oheData[[columnName+'2']]).toarray()
    dfIntermediate2[oneHotEnc.categories_[0]]=oheDataColumn2
    oheDataColumn1 = oneHotEnc.fit_transform(oheData[[columnName+'1']]).toarray()
    dfIntermediate1[oneHotEnc.categories_[0]]=oheDataColumn1
    
    #The OR operation is done as the features from the 3 columns are the data of the same categories and so a common set of columns for all the 3 segments will suffice
    dfIntermediate = (dfIntermediate3 | dfIntermediate2 | dfIntermediate1)
    
    #Dropping the feature NaN
    dfIntermediate = dfIntermediate.loc[:, dfIntermediate.columns.notna()]
    
    #Adding columnName as the prefix to the features to differentiate between the arrival and departure data
    dfIntermediate.columns=[columnName+'_'+iterator for iterator in dfIntermediate.columns]
    
    oheData[dfIntermediate.columns] = dfIntermediate
    
    #Replacing NaNs with 0
    oheData[dfIntermediate.columns] = oheData[dfIntermediate.columns].fillna(0).astype(int)
    return oheData

The categorical data have been coverted into features to be used in the prediction.

### pipeline function

In [ ]:
def process_data_gm(data, pipeline_functions):
    """Process the data for a guided model."""
    for function, arguments, keyword_arguments in pipeline_functions:
        if keyword_arguments and (not arguments):
            data = data.pipe(function, **keyword_arguments)
        elif (not keyword_arguments) and (arguments):
            data = data.pipe(function, *arguments)
        else:
            data = data.pipe(function)
    return data

def drop_columns(data, *columns):
    """Drop columns passed as arguments."""
    return data.drop(columns)


In [ ]:
#processing the data on Feature engineering functions
pipelines = [
    (oheCabinCodeColumns, None, None),
    (oheSegmentsAirportCode_AirlineName, None, {'columnName':'segmentsArrivalAirportCode'}),    
    (oheSegmentsAirportCode_AirlineName, None, {'columnName':'segmentsDepartureAirportCode'}),    
    (oheSegmentsAirportCode_AirlineName, None, {'columnName':'segmentsAirlineCode'}),    
    (drop_columns, ['Log Sale Price', 'Bedrooms'], None)
]

In [ ]:
data = process_data_gm(data, pipelines)

In [ ]:
data

### Splitting data for testing, training and validation

We are splitting the data into three segments for testing, training and validation in the proportion of 8:1:1.

In [ ]:
from sklearn.model_selection import train_test_split

# Let's say we want to split the data in 80:10:10 for train:valid:test dataset
train_size=0.8

X = data.drop(columns = ['baseFare']).copy()
y = data['baseFare']

# In the first step we will split the data in training and remaining dataset
X_train, X_rem, y_train, y_rem = train_test_split(X,y, train_size=0.8)

# Now since we want the valid and test size to be equal (10% each of overall data). 
# we have to define valid_size=0.5 (that is 50% of remaining data)
test_size = 0.5
X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5)
print(X_train.shape), print(y_train.shape)
print(X_valid.shape), print(y_valid.shape)
print(X_test.shape), print(y_test.shape)

### Prediction using Linear Regression without L2 Regularziation

In [ ]:
from sklearn import linear_model as lm

linear_model = lm.LinearRegression(fit_intercept=True) # creating the linear model

In [ ]:
y_fitted = linear_model.fit(X_train, y_train) #fitting the model on x_train, y_train
y_predicted_test = linear_model.predict(X_test) # prediction using x_train

### Prediction with L2 Regularziation

In this model we will be adding Regulariziation to combat overfitting if present

In [ ]:
from sklearn.linear_model import Ridge

ridge_model = Ridge(alpha = 10000) # creating the Regularized model

In [ ]:
y_fitted_l2= ridge_model.fit(X_train, y_train) #fitting the model on x_train, y_train
y_predicted_l2_test = ridge_model.predict(X_test_m1)# prediction using x_train
lasso_model.corf_

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

## Validation of model

### RMSE Value-Chirag

### plot of the model predictions versus the observations

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

## Inference and Conclusion

#### EDA Inference

For exploratory data analysis, we aim to find the airport with the highest traffic or the best-connected airports, the price of flights during the different hours of the day, the average/minimum travel distance for which people prefer flight, whether an average flight is fully booked/percentage of flights booked, and popular airlines.

#### Prediction Inference